In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
color = sns.color_palette()
%matplotlib inline
from wordcloud import WordCloud
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
import plotly.express as px
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
import re

In [ ]:
df = pd.read_csv('combined_csv.csv')
df.head()

In [ ]:
stocks = pd.read_csv('stock_screener.csv')
stock = stocks["Symbol"]
stock

In [ ]:
symbols = ['GME','AMC','BB','CLNE','CLOV','CLF','TLRY','NOK','TKAT','CRSR','OCGN','LFMD']

In [ ]:
symbols = stock.values.tolist()
symbols

In [ ]:
fig = px.histogram(df, x="Score")
fig.update_traces(marker_color="turquoise",marker_line_color='rgb(8,48,107)',
                  marker_line_width=1.5)
fig.update_layout(title_text='Comment Score')
fig.show()

In [ ]:
df_1 = df["Score"]
df_1

In [ ]:
filtered = df[df.Score > 30]

In [ ]:
fig = px.histogram(filtered, x="Score")
fig.update_traces(marker_color="turquoise",marker_line_color='rgb(8,48,107)',
                  marker_line_width=1.5)
fig.update_layout(title_text='Comment Score')
fig.show()

In [ ]:
# Create stopword list:
stopwords = set(stopwords.words('english'))
textt = " ".join(comment for comment in filtered.Comment)
wordcloud = WordCloud(stopwords=stopwords).generate(textt)
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.savefig('wordcloud11.png')
plt.show()

In [ ]:
# assign reviews with score > 3 as positive sentiment
# score < 3 negative sentiment
# remove score = 3
df = df[df['Score'] != 10]
df['sentiment'] = df['Score'].apply(lambda rating : +1 if rating > 10 else -1)

In [ ]:
df

In [ ]:
# split df - positive and negative sentiment:
positive = df[df['sentiment'] == 1]
negative = df[df['sentiment'] == -1]

In [ ]:
stopwords = stopwords
## good and great removed because they were included in negative sentiment
pos = " ".join(review for review in positive.Comment)
wordcloud2 = WordCloud(stopwords=stopwords).generate(pos)
plt.imshow(wordcloud2, interpolation='bilinear')
plt.axis("off")
plt.show()

In [ ]:
neg = " ".join(review for review in negative.Comment)
wordcloud3 = WordCloud(stopwords=stopwords).generate(neg)
plt.imshow(wordcloud3, interpolation='bilinear')
plt.axis("off")
plt.savefig('wordcloud33.png')
plt.show()

In [ ]:
def remove_punctuation(text):
    final = "".join(u for u in text if u not in ("?", ".", ";", ":",  "!",'"', "-"))
    return final
df['Comment'] = df['Comment'].apply(remove_punctuation)
df = df.dropna(subset=['Author'])
df['Author'] = df['Author'].apply(remove_punctuation)

In [ ]:
df

In [ ]:
dfNew = df[['Comment','sentiment']]
dfNew.head()

In [ ]:
# random split train and test data
index = df.index
dfNew['random_number'] = np.random.randn(len(index))
train = dfNew[dfNew['random_number'] <= 0.8]
test = dfNew[dfNew['random_number'] > 0.8]

In [ ]:
# count vectorizer:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(token_pattern=r'\b\w+\b')
train_matrix = vectorizer.fit_transform(train['Comment'])
test_matrix = vectorizer.transform(test['Comment'])

In [ ]:
# Logistic Regression
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()

In [ ]:
X_train = train_matrix
X_test = test_matrix
y_train = train['sentiment']
y_test = test['sentiment']

In [ ]:
lr.fit(X_train,y_train)

In [ ]:
predictions = lr.predict(X_test)

In [ ]:
# find accuracy, precision, recall:
from sklearn.metrics import confusion_matrix,classification_report
new = np.asarray(y_test)
confusion_matrix(predictions,y_test)

In [ ]:
print(classification_report(predictions,y_test))


In [ ]:
new = df[df['Comment'].isin([stock])]
new.head()

In [ ]:
df['tokenized_sents'] = df.apply(lambda column: nltk.word_tokenize(column['Comment']), axis=1)
df

In [ ]:
new = df[df['tokenized_sents'].isin([stock])]
new.head()

In [ ]:
search_list = symbols

df['symbols'] = df.Comment.str.extract('({0})'.format('|'.join(search_list)), flags=re.IGNORECASE)
result = df[~pd.isna(df.symbols)]

result

### Test Code

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
color = sns.color_palette()
%matplotlib inline
from wordcloud import WordCloud
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
import plotly.express as px
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
import re



In [ ]:
symbols = ['GME','AMC','BB','CLNE','CLOV','CLF','TLRY','NOK','TKAT','CRSR','OCGN','LFMD']

df = pd.read_csv('combined_csv.csv')
df.head()



In [ ]:
search_list = symbols

df['symbols'] = df.Comment.str.extract('({0})'.format('|'.join(search_list)), flags=re.IGNORECASE)
result = df[~pd.isna(df.symbols)]

result



In [ ]:
# Create stopword list:
stopwords = set(stopwords.words('english'))
textt = " ".join(comment for comment in result.Comment)
wordcloud = WordCloud(stopwords=stopwords).generate(textt)
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.savefig('wordcloud11.png')
plt.show()



In [ ]:
# assign reviews with score > 3 as positive sentiment
# score < 3 negative sentiment
# remove score = 3
result = result[result['Score'] != 10]
result['sentiment'] = result['Score'].apply(lambda rating : +1 if rating > 10 else -1)



In [ ]:
dfNew = result[['Comment','sentiment']]
dfNew.head()



In [ ]:
# random split train and test data
index = result.index
dfNew['random_number'] = np.random.randn(len(index))
train = dfNew[dfNew['random_number'] <= 0.8]
test = dfNew[dfNew['random_number'] > 0.8]



In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(token_pattern=r'\b\w+\b')
train_matrix = vectorizer.fit_transform(train['Comment'])
test_matrix = vectorizer.transform(test['Comment'])



In [ ]:
# Logistic Regression
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()



In [ ]:
X_train = train_matrix
X_test = test_matrix
y_train = train['sentiment']
y_test = test['sentiment']



In [ ]:
lr.fit(X_train,y_train)



In [ ]:
predictions = lr.predict(X_test)



In [ ]:
# find accuracy, precision, recall:
from sklearn.metrics import confusion_matrix,classification_report
new = np.asarray(y_test)
confusion_matrix(predictions,y_test)



In [ ]:
print(classification_report(predictions,y_test))

